In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

In [3]:
datadf

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat_orig,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",1.0,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,"CHI, INTERACT",depending on the professional values,7.0,7.0,"any, depending on the values",5.0,0,0,0,4,2,2,2,3,2,2,2,NaN,research-practice connections,NaN,research-practice connections,2
288,Academic,4,1.0,1,2,1,3,3,3,3,3,0,3,4.0,3.0,Higher education,Multiple,Multiple,5,5,5,5,5,5,5,5,2,5,"Yes, I was taught how to conduct research",1.0,Research methods and statistics training in bo...,2,3,2,2,2,2,3,3,2,3,2,2,2,3,3,4,3,4,2,3,3,2,4,"Content, cost, timing (can I get away at this ...","Dagstuhl workshop - small, intense, intimate g...",Hawaii international conference on Education,I would attend any of these,6.0,6.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,3,3,3,3,3,1,1,2,NaN,NaN,NaN,Don’t create schedule conflicts with other rel...,2
294,Academic,4,1.0,3,3,1,3,1,3,3,3,1,3,4.0,3.0,University,"User research / Design research\n\nInterviews,...",Online FG,4,4,3,3,4,4,4,4,3,3,"Yes, I was taught how to conduct research",1.0,BA Sociology\nMA LIS\nPhD. LIS\n\nonline cours...,2,3,2,2,2,3,4,3,3,3,3,3,3,3,3,4,4,3,4,3,3,4,3,- time for travel (having two small children]\...,Courses at Copenhagen University (deep knowle...,"Research Tuesdays (Czech), UX/Design Conferen...",It depends - I really enjoyed small conference...,7.0,7.0,It depends,5.0,0,0,0,2,2,2,2,2,2,2,2,NaN,NaN,NaN,Looking forward! \n,2
281,Academic,4,1.0,3,4,0,2,3,2,0,1,0,0,2.0,1.0,Ed Tech,"surveys, interviews, focus groups, secondary d...","surveys, interviews, focus groups, secondary d...",1,1,0,1,1,1,0,1,0,0,"Yes, I was taught how to conduct research",1.0,"I have an Applied Anthropology, MA. I designed...",1,2,0,1,2,3,4,3,2,4,2,3,3,2,3,3,3,1,1,1,2,3,4,My professional development budget allows me t...,A workshop at the Society for Applied Anthropo...,A local education implementation conference.,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,2,3,2,NaN,"Research design, innovative methods, communica...",NaN,NaN,2
280,Academic,4,1.0,3,3,0,1,1,3,2,1,0,0,NaN,NaN,education,NaN,NaN,4,4,0,2,2,3,2,3,0,0,"No, I was not taught how to conduct research",0.0,NaN,1,1,1,1,2,3,3,3,1,2,1,2,1,3,1,4,2,3,1,2,2,4,4,NaN,NaN,NaN,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,1,2,2,You should consider the combination of confere...,NaN,

In [4]:
datadf.events_attend_recent.dtype

dtype('O')

In [5]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,0.625344,0.334711,0.061983,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,0.484368,0.472215,0.241292,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,1.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.00000

In [6]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 78 columns):
job_title                         726 non-null category
job_taxo                          726 non-null category
job_id                            723 non-null category
job_conduct_res                   726 non-null int64
job_analyze_res                   726 non-null int64
job_buy_res_report                726 non-null int64
job_manage_res_proj               726 non-null int64
job_observe_res                   726 non-null int64
job_plan_res                      726 non-null int64
job_teach_res                     726 non-null int64
job_advocate_res                  726 non-null int64
job_hire_res_vendor               726 non-null int64
job_lead_res_team                 726 non-null int64
num_employees                     723 non-null category
num_researchers                   722 non-null category
primary_industry                  726 non-null category
types_res_used               

## 5. What is your company or organization's primary industry?

In [7]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

In [117]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.4)
primary_industry_matrix, primary_industry_vector

(<726x180 sparse matrix of type '<class 'numpy.int64'>'
 	with 1239 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.4, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [121]:
lda5 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [122]:
lda5.fit(primary_industry_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [123]:
pyLDAvis.sklearn.prepare(lda5, primary_industry_matrix, primary_industry_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.158562 -0.034322       1        1  28.206163
2      0.147048 -0.143685       2        1  27.847899
3     -0.285705 -0.113296       3        1  22.431967
0     -0.019906  0.291303       4        1  21.513971, topic_info=    Category       Freq                  Term      Total  loglift  logprob
75   Default  47.000000            healthcare  47.000000  30.0000  30.0000
151  Default  39.000000              software  39.000000  29.0000  29.0000
157  Default  40.000000                  tech  40.000000  28.0000  28.0000
159  Default  39.000000            technology  39.000000  27.0000  27.0000
67   Default  37.000000               fintech  37.000000  26.0000  26.0000
51   Default  37.000000             education  37.000000  25.0000  25.0000
26   Default  34.000000            consulting  34.000000  24.0000  24.0000
142  Default  23.000000                retail  23.000000  23.0000  23.0000
65   Default  25.000000             financial  25.000000  22.0000  22.0000
148  Default  38.000000               service  38.000000  21.0000  21.0000
136  Default  22.000000              research  22.000000  20.0000  20.0000
71   Default  30.000000            government  30.000000  19.0000  19.0000
143  Default  15.000000                  saas  15.000000  18.0000  18.0000
92   Default  16.000000             insurance  16.000000  17.0000  17.0000
66   Default  19.000000     financial service  19.000000  16.0000  16.0000
63   Default  13.000000               finance  13.000000  15.0000  15.0000
47   Default  16.000000             ecommerce  16.000000  14.0000  14.0000
39   Default  30.000000                design  30.000000  13.0000  13.0000
43   Default  13.000000               digital  13.000000  12.0000  12.0000
86   Default  22.000000              industry  22.000000  11.0000  11.0000
128  Default  10.000000                public  10.000000  10.0000  10.0000
146  Default   9.000000                sector   9.000000   9.0000   9.0000
118  Default   9.000000             nonprofit   9.000000   8.0000   8.0000
55   Default   9.000000            enterprise   9.000000   7.0000   7.0000
163  Default   8.000000        transportation   8.000000   6.0000   6.0000
2    Default  15.000000                agency  15.000000   5.0000   5.0000
133  Default   9.000000                  real   9.000000   4.0000   4.0000
134  Default   9.000000           real estate   9.000000   3.0000   3.0000
58   Default   9.000000                estate   9.000000   2.0000   2.0000
77   Default   9.000000                higher   9.000000   1.0000   1.0000
..       ...        ...                   ...        ...      ...      ...
128   Topic4   9.502355                public  10.148601   1.4707  -3.3405
146   Topic4   8.564502                sector   9.220428   1.4627  -3.4444
107   Topic4   6.716552             marketing   7.366489   1.4441  -3.6875
74    Topic4   6.715947                health   7.366552   1.4440  -3.6875
160   Topic4   4.867463               telecom   5.512451   1.4120  -4.0095
129   Topic4   4.865846         public sector   5.512315   1.4117  -4.0098
154   Topic4   3.940669      software service   4.585234   1.3850  -4.2207
81    Topic4   3.940550                    hr   4.585225   1.3849  -4.2207
82    Topic4   3.013411               hr tech   3.657982   1.3426  -4.4890
52    Topic4   3.013385           electronics   3.657980   1.3426  -4.4890
29    Topic4   3.013385  consumer electronics   3.657980   1.3426  -4.4890
97    Topic4   3.013171                 legal   3.657965   1.3426  -4.4890
34    Topic4   3.012883              customer   3.657947   1.3425  -4.4891
140   Topic4   3.009847              resource   3.657666   1.3415  -4.4901
69    Topic4   3.009190             freelance   3.657719   1.3413  -4.4904
4     Topic4   3.001796            automotive   3.656965   1.3390  -4.4928
28    Topic4   6.589999          

#### Topic groups 

1. Education - 28%. Top 5 following words: service, government, financial, research and design
2. Fintech - 28%. Top 5 following words: consulting, industry, digital, agency, real estate
3. Healthcare 22% Top 5 following words: technology, retail, insurance, finance, nonprofit
4. Tech 22%: Top 5 following words: software, saas, public, sector, service


In [12]:
lda_W5 = lda5.transform(primary_industry_matrix)

In [13]:
top_doc_column5 = datadf.primary_industry.dropna()

In [14]:
word_count_matrix5, count_vect5 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [15]:
LDA5a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA5a.fit(word_count_matrix5)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [16]:
lda_H = LDA5a.transform(word_count_matrix5)

In [17]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column5, 3)

Top 3 Documents for Topic 0: 

Document 1
real estate tech

Document 2
real estate tech

Document 3
sotware development across domain such a data science deep learning health tech interaction etc

Top 3 Documents for Topic 1: 

Document 1
design research service and innovation planning aka design planning design ledstrategy

Document 2
market research agency 95 client are in financial service

Document 3
ux research we're a ux research agency a vendor used by fortune 500 company

Top 3 Documents for Topic 2: 

Document 1
commercial real estate

Document 2
commercial real estate

Document 3
architecture engineering construction aec industry manufacturing design medium entertainment industry

Top 3 Documents for Topic 3: 

Document 1
banking retail edu logistic government entertainment tourism aviation

Document 2
financial sector public sector resource technology retail healthcare

Document 3
aerospaceautomativebankingcentral government chemical consumer package goodsdefenceeducation ma

## 6. What types of research do you currently use to make decisions?

In [18]:
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.basic_clean)
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.lemmatize)

In [136]:
types_res_used_matrix, types_res_used_vector = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), ngram=(1,3), max_df=.4)
types_res_used_matrix, types_res_used_vector

(<690x1212 sparse matrix of type '<class 'numpy.int64'>'
 	with 8308 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.4, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [140]:
lda6 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [141]:
lda6.fit(types_res_used_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [142]:
pyLDAvis.sklearn.prepare(lda6, types_res_used_matrix, types_res_used_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.106207  0.0       1        1  52.062178
0     -0.106207  0.0       2        1  47.937822, topic_info=     Category        Freq                         Term       Total  loglift  \
659   Default  147.000000                  qualitative  147.000000  30.0000   
693   Default   77.000000                 quantitative   77.000000  29.0000   
962   Default   85.000000                         test   85.000000  28.0000   
67    Default   61.000000                         card   61.000000  27.0000   
656   Default   42.000000                         qual   42.000000  26.0000   
369   Default   44.000000                      inquiry   44.000000  25.0000   
442   Default   36.000000          interview usability   36.000000  24.0000   
102   Default   42.000000                      concept   42.000000  23.0000   
123   Default   43.000000           contextual inquiry   43.000000  22.0000   
510   Default   52.000000                       method   52.000000  21.0000   
727   Default  329.000000                     research  329.000000  20.0000   
835   Default   36.000000                         sort   36.000000  19.0000   
236   Default   40.000000                   evaluative   40.000000  18.0000   
68    Default   35.000000                    card sort   35.000000  17.0000   
1105  Default   37.000000               usability test   37.000000  16.0000   
1062  Default   25.000000                         tree   25.000000  15.0000   
73    Default   25.000000                 card sorting   25.000000  14.0000   
845   Default   25.000000                      sorting   25.000000  13.0000   
678   Default   28.000000         qualitative research   28.000000  12.0000   
673   Default   27.000000     qualitative quantitative   27.000000  11.0000   
657   Default   21.000000                   qual quant   21.000000  10.0000   
686   Default   48.000000                        quant   48.000000   9.0000   
330   Default   20.000000                    heuristic   20.000000   8.0000   
805   Default   22.000000                    secondary   22.000000   7.0000   
106   Default   22.000000              concept testing   22.000000   6.0000   
445   Default   19.000000  interview usability testing   19.000000   5.0000   
792   Default   19.000000                       review   19.000000   4.0000   
296   Default   42.000000                   generative   42.000000   3.0000   
530   Default   18.000000                        mixed   18.000000   2.0000   
297   Default   20.000000        generative evaluative   20.000000   1.0000   
...       ...         ...                          ...         ...      ...   
689    Topic2    7.397837                   quant qual    7.908860   0.6685   
391    Topic2    6.573392       interview card sorting    7.030093   0.6681   
835    Topic2   33.516410                         sort   36.028887   0.6630   
68     Topic2   32.642348                    card sort   35.150022   0.6613   
962    Topic2   77.469440                         test   85.235470   0.6397   
1105   Topic2   34.412350               usability test   37.784935   0.6418   
510    Topic2   47.036213                       method   52.721227   0.6212   
564    Topic2    7.380416        observation interview    7.908826   0.6661   
1210   Topic2   28.360916                     workshop   33.387289   0.5721   
686    Topic2   39.429723                        quant   48.320511   0.5319   
177    Topic2   13.870660                desk research   15.815894   0.6040   
647    Topic2   18.360235                    prototype   21.964733   0.5560   
984    Topic2  152.362808                      testing  266.074014   0.1778   
176    Topic2   15.182333                         desk   17.572762   0.5890   
1115   Topic2   87.957498            usability testing  148.410195   0.2121   
900    Topic2  118.699203                       

#### Topic groups 

1. Qualitative (the top word was actually 'research', which we excluded)
2. Testing 


In [23]:
lda_W6 = lda6.transform(types_res_used_matrix)

In [24]:
top_doc_column6 = datadf.types_res_used.dropna()

In [25]:
word_count_matrix6, count_vect6 = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [26]:
LDA6a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA6a.fit(word_count_matrix6)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [27]:
lda_H = LDA6a.transform(word_count_matrix6)

In [28]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column6, 3)

Top 3 Documents for Topic 0: 

Document 1
qualitative interview survey card sort prototype testing usability testing analytics etc

Document 2
usability testing moderated and unmoderated card sort tree jacking max diff jtbd odi survey observational ethnographic research concept validation others a needed

Document 3
usability testing empathy1x1 interview survey card sort tree test prototype and concept testing diary study eye tracking

Top 3 Documents for Topic 1: 

Document 1
user research participant observation interview contextual enquiry usability testing feedback analysis data analysis survey

Document 2
contextual inquiry interview survey feedback form quant behavior analysis quant usability study usability testing ux workshop

Document 3
foundational product research usability testing 11 interview unmoderated user testing using usertestingcom survey contextual inquiry diary study analytics

Top 3 Documents for Topic 2: 

Document 1
qual research interview in lab moderated usabi

## 7. What types of research are you considering in the future?

In [29]:
datadf.future_res = datadf.future_res.dropna().apply(nlp.basic_clean)
datadf.future_res = datadf.future_res.dropna().apply(nlp.lemmatize)

In [30]:
future_res_matrix, future_res_vector = nlp.create_wordcount_matrix(datadf.future_res.dropna(), ngram=(1,3), max_df=.4)
future_res_matrix, future_res_vector

(<609x540 sparse matrix of type '<class 'numpy.int64'>'
 	with 3008 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [146]:
lda7 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [144]:
lda7.fit(future_res_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [145]:
pyLDAvis.sklearn.prepare(lda7, future_res_matrix, future_res_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.011066 -0.162547       1        1  30.652362
3      0.020336 -0.008149       2        1  26.605634
1     -0.188220  0.087641       3        1  23.924497
2      0.178950  0.083055       4        1  18.817507, topic_info=    Category        Freq                   Term       Total  loglift  logprob
371  Default  138.000000               research  138.000000  30.0000  30.0000
434  Default   69.000000                  study   69.000000  29.0000  29.0000
90   Default   39.000000                  diary   39.000000  28.0000  28.0000
459  Default   91.000000                testing   91.000000  27.0000  27.0000
91   Default   35.000000            diary study   35.000000  26.0000  26.0000
343  Default   36.000000           quantitative   36.000000  25.0000  25.0000
76   Default   43.000000                   data   43.000000  24.0000  24.0000
0    Default   20.000000                     ab   20.000000  23.0000  23.0000
506  Default   32.000000      usability testing   32.000000  22.0000  22.0000
2    Default   19.000000             ab testing   19.000000  21.0000  21.0000
12   Default   31.000000              analytics   31.000000  20.0000  20.0000
85   Default   28.000000                 design   28.000000  19.0000  19.0000
360  Default   29.000000                 remote   29.000000  18.0000  18.0000
330  Default   26.000000            qualitative   26.000000  17.0000  17.0000
275  Default   17.000000                    new   17.000000  16.0000  16.0000
481  Default   11.000000               tracking   11.000000  15.0000  15.0000
299  Default    8.000000          participatory    8.000000  14.0000  14.0000
443  Default    8.000000                   sure    8.000000  13.0000  13.0000
61   Default   22.000000             contextual   22.000000  12.0000  12.0000
522  Default    9.000000                  using    9.000000  11.0000  11.0000
501  Default   53.000000              usability   53.000000  10.0000  10.0000
154  Default    7.000000             generative    7.000000   9.0000   9.0000
10   Default   22.000000               analysis   22.000000   8.0000   8.0000
25   Default   10.000000           benchmarking   10.000000   7.0000   7.0000
491  Default   13.000000                   type   13.000000   6.0000   6.0000
168  Default   10.000000                   help   10.000000   5.0000   5.0000
347  Default    9.000000  quantitative research    9.000000   4.0000   4.0000
407  Default    9.000000                session    9.000000   3.0000   3.0000
494  Default   29.000000            unmoderated   29.000000   2.0000   2.0000
444  Default   37.000000                 survey   37.000000   1.0000   1.0000
..       ...         ...                    ...         ...      ...      ...
338   Topic4    2.710536             quant data    3.353705   1.4575  -5.3781
161   Topic4    2.710314                   good    3.353672   1.4574  -5.3781
410   Topic4    2.709932                similar    3.353676   1.4572  -5.3783
169   Topic4    2.708774            help inform    3.353787   1.4568  -5.3787
142   Topic4    2.707678              formative    3.353740   1.4564  -5.3791
432   Topic4    2.707348               stronger    3.353768   1.4563  -5.3792
376   Topic4    2.691751        research design    3.353912   1.4504  -5.3850
396   Topic4    5.903591                science    7.546364   1.4249  -4.5996
147   Topic4    4.398468                 future    5.874220   1.3811  -4.8939
190   Topic4    1.877268                   info    2.519299   1.3762  -5.7454
379   Topic4    1.877255       research finding    2.519299   1.3762  -5.7454
397   Topic4    1.877255              scientist    2.519299   1.3762  -5.7454
80    Topic4    1.877255         data scientist    2.519299   1.3762  -5.7454
168   Topic4    7.620338                   help   10.943948   1.3084  -4.3444
380   Topic4    3.552830        research meth

#### Topic groups 

1. Usability
2. Quantitative
3. Diary

In [34]:
lda_W7 = lda7.transform(future_res_matrix)

In [35]:
top_doc_column7 = datadf.future_res.dropna()

In [36]:
word_count_matrix7, count_vect7 = nlp.create_wordcount_matrix(datadf.future_res.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [37]:
LDA7a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA7a.fit(word_count_matrix7)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [38]:
lda_H = LDA7a.transform(word_count_matrix7)

In [39]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column7, 3)


Top 3 Documents for Topic 0: 

Document 1
user experience research workshop survey interview usability testing benchmark card sorting etc and i would like to delve deeper in analytics

Document 2
doing more strategic piece to help inform the product strategy early on before the okrs are set have fundamental understanding our user in different market since we have a global product and we havent done a lot of thinking around localized content and market difference doing information architecture research

Document 3
abn multivariate testing analytic research behavioural research card sortingtree testing content auditing experience mapping information architecture ia lab testing build persona development research participant recruitment management scenariotask mapping survey design analysis usability testing userstakeholder interview workshop

Top 3 Documents for Topic 1: 

Document 1
much more usability testing heatmappingbehavior tracking along the line of what pendoio offer exploratory 

## 10. Describe your educational background with research

In [40]:
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.basic_clean)
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.lemmatize)

In [41]:
research_educ_desc_matrix, research_educ_desc_vector = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), ngram=(1,3), max_df=.5)
research_educ_desc_matrix, research_educ_desc_vector

(<540x1119 sparse matrix of type '<class 'numpy.int64'>'
 	with 6861 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [147]:
lda10 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [148]:
lda10.fit(research_educ_desc_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [149]:
pyLDAvis.sklearn.prepare(lda10, research_educ_desc_matrix, research_educ_desc_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.091042  0.0       1        1  59.967267
0     -0.091042  0.0       2        1  40.032733, topic_info=     Category        Freq                       Term       Total  loglift  \
590   Default  129.000000                     master  129.000000  30.0000   
235   Default  101.000000                     degree  101.000000  29.0000   
594   Default   39.000000              master degree   39.000000  28.0000   
1052  Default   67.000000                         ux   67.000000  27.0000   
422   Default   40.000000                        hci   40.000000  26.0000   
439   Default   47.000000                      human   47.000000  25.0000   
945   Default   74.000000                      study   74.000000  24.0000   
493   Default   29.000000                interaction   29.000000  23.0000   
585   Default   15.000000                     market   15.000000  22.0000   
586   Default   14.000000            market research   14.000000  21.0000   
532   Default   27.000000                    learned   27.000000  20.0000   
167   Default   12.000000                   computer   12.000000  19.0000   
819   Default   46.000000            research method   46.000000  18.0000   
285   Default   24.000000            design research   24.000000  17.0000   
1064  Default   22.000000                ux research   22.000000  16.0000   
453   Default   11.000000              humancomputer   11.000000  15.0000   
454   Default   11.000000  humancomputer interaction   11.000000  14.0000   
904   Default   23.000000                  sociology   23.000000  13.0000   
678   Default   24.000000               participated   24.000000  12.0000   
478   Default   22.000000                information   22.000000  11.0000   
53    Default   11.000000                       area   11.000000  10.0000   
330   Default   16.000000                engineering   16.000000   9.0000   
444   Default   30.000000               human factor   30.000000   8.0000   
402   Default   29.000000                       grad   29.000000   7.0000   
679   Default   10.000000       participated running   10.000000   6.0000   
322   Default   22.000000                  education   22.000000   5.0000   
852   Default   21.000000                 researcher   21.000000   4.0000   
938   Default   24.000000                    studied   24.000000   3.0000   
865   Default   10.000000              running study   10.000000   2.0000   
403   Default   24.000000                grad school   24.000000   1.0000   
...       ...         ...                        ...         ...      ...   
422    Topic2   32.739497                        hci   40.118456   0.7122   
702    Topic2    6.821567                  phd human    7.658460   0.7998   
235    Topic2   76.348229                     degree  101.858546   0.6272   
639    Topic2    7.516438                methodology    8.512118   0.7911   
493    Topic2   23.577273                interaction   29.910054   0.6776   
330    Topic2   13.398873                engineering   16.211043   0.7250   
439    Topic2   35.393377                      human   47.956494   0.6117   
945    Topic2   47.929234                      study   74.807909   0.4703   
904    Topic2   17.328354                  sociology   23.110519   0.6275   
678    Topic2   18.115681               participated   24.843792   0.5996   
71     Topic2   14.223073                   bachelor   18.826326   0.6351   
938    Topic2   17.081636                    studied   24.004749   0.5752   
402    Topic2   19.822821                       grad   29.188229   0.5285   
444    Topic2   20.640322               human factor   30.920202   0.5113   
403    Topic2   16.891295                grad school   24.012950   0.5637   
862    Topic2   15.313428                    running   21.430141   0.5794   
359    Topic2   20.409791                     factor   31.813734 

#### Topic groups 

1. UX
2. Market


In [45]:
lda_W10 = lda10.transform(research_educ_desc_matrix)

In [46]:
top_doc_column10 = datadf.research_educ_desc.dropna()

In [47]:
word_count_matrix10, count_vect10 = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [48]:
LDA10a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA10a.fit(word_count_matrix10)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [49]:
lda_H = LDA10a.transform(word_count_matrix10)

In [50]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column10, 3)

Top 3 Documents for Topic 0: 

Document 1
i took professional development course in research up to one year in length at undergraduate level and then started postgraduate level private tuition i'd love to do a master or phd degree but am not able to manage the commitment even parttime alongside working full time i continue to take short course to increase my knowledge and develop my skill

Document 2
phd in human computer interaction research method data analysis coursework in various area eg ethnography statistic a well a supporting other people's research and eventually running my own research with supervision and advice from more senior people

Document 3
undergraduate degree in psychology extensive study of behavioral and scientific research methodology and analysis qual and quant interned and worked in lab for several year genetics neuroethology biopsychology graduate degree in architecture extensive study and practice of many type of research method

Top 3 Documents for Topic 1: 

## 14. How do you decide which events to attend?

In [51]:
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.basic_clean)
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.lemmatize)

In [151]:
how_pick_events_matrix, how_pick_events_vector = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), ngram=(1,3), max_df=.4)
how_pick_events_matrix, how_pick_events_vector

(<644x1332 sparse matrix of type '<class 'numpy.int64'>'
 	with 7899 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.4, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [153]:
lda14 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [154]:
lda14.fit(how_pick_events_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [155]:
pyLDAvis.sklearn.prepare(lda14, how_pick_events_matrix, how_pick_events_vector)

PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
1      0.09293  0.0       1        1  53.676519
0     -0.09293  0.0       2        1  46.323481, topic_info=     Category        Freq             Term       Total  loglift  logprob
206   Default  194.000000       conference  194.000000  30.0000  30.0000
1155  Default  165.000000            topic  165.000000  29.0000  29.0000
262   Default  144.000000             cost  144.000000  28.0000  28.0000
71    Default  101.000000           attend  101.000000  27.0000  27.0000
1324  Default   51.000000             year   51.000000  26.0000  26.0000
684   Default   42.000000            local   42.000000  25.0000  25.0000
133   Default   57.000000           budget   57.000000  24.0000  24.0000
1158  Default   24.000000    topic covered   24.000000  23.0000  23.0000
990   Default   46.000000         research   46.000000  22.0000  22.0000
1190  Default  134.000000           travel  134.000000  21.0000  21.0000
292   Default   29.000000          covered   29.000000  20.0000  20.0000
105   Default   45.000000            based   45.000000  19.0000  19.0000
1151  Default   25.000000           timing   25.000000  18.0000  18.0000
328   Default   17.000000          depends   17.000000  17.0000  17.0000
911   Default   40.000000            price   40.000000  16.0000  16.0000
569   Default   17.000000               im   17.000000  15.0000  15.0000
784   Default   23.000000       networking   23.000000  14.0000  14.0000
973   Default   13.000000        relevance   13.000000  13.0000  13.0000
1014  Default   24.000000         schedule   24.000000  12.0000  12.0000
1073  Default   13.000000    speaker topic   13.000000  11.0000  11.0000
1218  Default   27.000000              try   27.000000  10.0000  10.0000
245   Default   20.000000    consideration   20.000000   9.0000   9.0000
239   Default   16.000000  conference year   16.000000   8.0000   8.0000
280   Default   12.000000      cost travel   12.000000   7.0000   7.0000
354   Default   12.000000         distance   12.000000   6.0000   6.0000
1281  Default   30.000000             want   30.000000   5.0000   5.0000
943   Default   18.000000          quality   18.000000   4.0000   4.0000
333   Default   14.000000           design   14.000000   3.0000   3.0000
322   Default   11.000000         decision   11.000000   2.0000   2.0000
1255  Default   16.000000               ux   16.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
575    Topic2    8.516176           impact    9.394761   0.6713  -6.1308
1155   Topic2  137.941355            topic  165.136236   0.5896  -3.3459
262    Topic2  114.035571             cost  144.778160   0.5308  -3.5362
245    Topic2   17.786645    consideration   20.518475   0.6266  -5.3943
105    Topic2   37.475565            based   45.358901   0.5786  -4.6491
133    Topic2   46.712350           budget   57.357862   0.5642  -4.4287
1014   Topic2   20.848439         schedule   24.810032   0.5956  -5.2355
943    Topic2   16.106512          quality   18.813790   0.6142  -5.4935
911    Topic2   31.379754            price   40.272511   0.5200  -4.8266
1190   Topic2   90.382494           travel  134.905817   0.3690  -3.7687
1133   Topic2   14.366558           ticket   17.110673   0.5947  -5.6079
1281   Topic2   23.724724             want   30.855178   0.5067  -5.1062
483    Topic2   11.143293              fit   12.823349   0.6291  -5.8619
460    Topic2   24.350180           factor   32.587509   0.4781  -5.0802
695    Topic2   64.105560         location  107.614991   0.2515  -4.1122
191    Topic2   31.981637          company   47.251638   0.3792  -4.8076
1301   Topic2   45.635564             work   80.119525   0.2067  -4.4521
1136   Topic2   47.541748             time   86.190099   0.1746  -4.4111
1064   Topic2   49.309648          speaker   93.138662   0.1336  -4.3746
632    Topic2   15.230718             know

#### Topic groups 

1. Local
2. Topic


In [56]:
lda_W14 = lda14.transform(how_pick_events_matrix)

In [57]:
top_doc_column14 = datadf.how_pick_events.dropna()

In [58]:
word_count_matrix14, count_vect14 = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [59]:
LDA14a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA14a.fit(word_count_matrix14)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [60]:
lda_H = LDA14a.transform(word_count_matrix14)

In [61]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column14, 3)

Top 3 Documents for Topic 0: 

Document 1
it is usually a combination of date and my availability to travel the location the size the speaker list and whether the event seems likely to cater to more senior researcher or designer i'm at a point where traveling far for 101 level talk is just not useful to me

Document 2
depends on the cost of the event eg is it an evening meetup v a conference if it's the latter then it would depend on company training budget or depending on how relevant the topic are for my personal development goal also depending on the talk itself eg what the topic are which company are presenting i am more interested in advanced topic around strategy prioritisation or maybe new method

Document 3
location is the primary factor to consider because it will indicate whether i can afford to attend or not my company will cover cost of entry to a conference or workshop but will not cover travel or accommodation i also use personal vacation time to attend learning session s

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [62]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [156]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.4)
best_event_matrix, best_event_vector

(<594x1631 sparse matrix of type '<class 'numpy.int64'>'
 	with 9198 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.4, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [160]:
lda15 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [161]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [162]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.073775  0.0       1        1  64.038274
1     -0.073775  0.0       2        1  35.961726, topic_info=     Category        Freq          Term       Total  loglift  logprob
1438  Default   97.000000            ux   97.000000  30.0000  30.0000
1592  Default  134.000000      workshop  134.000000  29.0000  29.0000
48    Default   17.000000      activity   17.000000  28.0000  28.0000
118   Default   24.000000      attendee   24.000000  27.0000  27.0000
303   Default   36.000000        course   36.000000  26.0000  26.0000
559   Default   47.000000          good   47.000000  25.0000  25.0000
597   Default   31.000000         group   31.000000  24.0000  24.0000
774   Default   75.000000      learning   75.000000  23.0000  23.0000
1069  Default   11.000000       product   11.000000  22.0000  22.0000
829   Default   48.000000           lot   48.000000  21.0000  21.0000
966   Default   10.000000        online   10.000000  20.0000  20.0000
1229  Default   36.000000         small   36.000000  19.0000  19.0000
1627  Default    9.000000         young    9.000000  18.0000  18.0000
886   Default   10.000000        method   10.000000  17.0000  17.0000
376   Default   21.000000     different   21.000000  16.0000  16.0000
1472  Default    9.000000   ve attended    9.000000  15.0000  15.0000
350   Default   78.000000        design   78.000000  14.0000  14.0000
292   Default   10.000000  conversation   10.000000  13.0000  13.0000
1333  Default   76.000000          talk   76.000000  12.0000  12.0000
1113  Default   24.000000          real   24.000000  11.0000  11.0000
674   Default    8.000000          indi    8.000000  10.0000  10.0000
446   Default   74.000000         event   74.000000   9.0000   9.0000
196   Default    8.000000           chi    8.000000   8.0000   8.0000
1618  Default   47.000000          year   47.000000   7.0000   7.0000
1147  Default    8.000000      remember    8.000000   6.0000   6.0000
466   Default   74.000000    experience   74.000000   5.0000   5.0000
208   Default    8.000000         clear    8.000000   4.0000   4.0000
1007  Default   77.000000        people   77.000000   3.0000   3.0000
1393  Default   58.000000         topic   58.000000   2.0000   2.0000
248   Default  184.000000    conference  184.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
334    Topic2    8.223818  day workshop   10.061980   0.8210  -6.0707
597    Topic2   20.979208         group   31.257889   0.6240  -5.1342
376    Topic2   15.057778     different   21.948171   0.6459  -5.4659
1592   Topic2   61.704777      workshop  134.091507   0.2466  -4.0554
559    Topic2   27.353935          good   47.543798   0.4699  -4.8689
829    Topic2   26.490222           lot   48.464904   0.4187  -4.9010
1229   Topic2   20.793133         small   36.518085   0.4595  -5.1432
1350   Topic2    7.498494      teaching    9.225615   0.8154  -6.1631
1333   Topic2   33.808048          talk   76.951156   0.2002  -4.6571
446    Topic2   32.137698         event   74.415097   0.1831  -4.7077
1007   Topic2   32.694285        people   77.885597   0.1547  -4.6906
1393   Topic2   26.445717         topic   58.967824   0.2208  -4.9027
577    Topic2   41.157480         great  116.811313  -0.0204  -4.4604
611    Topic2   13.699140          hand   22.895671   0.5091  -5.5604
136    Topic2   20.127424          best   41.803860   0.2918  -5.1757
1483   Topic2   70.326382            wa  318.266333  -0.4870  -3.9246
1364   Topic2   16.717936         thing   31.485137   0.3897  -5.3613
1587   Topic2   20.646754          work   44.401301   0.2570  -5.1502
329    Topic2   24.492898           day   59.071965   0.1423  -4.9794
1152   Topic2   29.276769      research   85.943258  -0.0542  -4.8010
1511   Topic2   14.274138      wa great   26.365189   0.4091  -5.5193
1253   Topic2   23.677768       speaker   74.866254

#### Topic groups 

1. UX
2. Online


In [67]:
lda_W15 = lda15.transform(best_event_matrix)

In [68]:
top_doc_column15 = datadf.best_event.dropna()

In [69]:
word_count_matrix15, count_vect15 = nlp.create_wordcount_matrix(datadf.best_event.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [70]:
LDA15a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA15a.fit(word_count_matrix15)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [71]:
lda_H = LDA15a.transform(word_count_matrix15)

In [72]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column15, 3)

Top 3 Documents for Topic 0: 

Document 1
two polar opposite experience c2mtl highly produced inspiring talk specific networking and meeting discussion and opportunity in an adjacent area to my own not my core sme so not the same old people i used what i learned from that experience to design an incredible offsite for my organization a few month after i returned a 2day immersive workshopclass on analyzing data that i took from iit many year ago it wa full of tool some of which were new to me and handson feedback not only did i walk away with tool and skill that i didn't have before i used that experience to design immersive learning course in the future it wa one of the earliest learnbydoing class that i'd taken

Document 2
the best experience i had not ux focussed wa the impact factory training workshop that wa over 2 day called 'train the trainer' fantastic and engaging trainer and really interactive session with no slide deck at all i liked the ucd london conference in 2010 or 2011 

## 16. What if any events have you attended on the subject of research in the past few years?

In [97]:
datadf.events_attend_recent = datadf.events_attend_recent.astype('str')

In [98]:
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.basic_clean)
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.lemmatize)

In [166]:
events_attend_recent_matrix, events_attend_recent_vector = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), ngram=(1,3), max_df=.3)
events_attend_recent_matrix, events_attend_recent_vector

(<726x641 sparse matrix of type '<class 'numpy.int64'>'
 	with 3538 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [167]:
lda16 = LatentDirichletAllocation(n_components= 5, random_state = 42)

In [168]:
lda16.fit(events_attend_recent_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [169]:
pyLDAvis.sklearn.prepare(lda16, events_attend_recent_matrix, events_attend_recent_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.040672 -0.010639       1        1  25.755274
2      0.167709 -0.105920       2        1  21.540317
3     -0.026678  0.011488       3        1  20.920277
1     -0.183202 -0.099325       4        1  20.630694
0      0.001499  0.204395       5        1  11.153437, topic_info=    Category        Freq                   Term       Total  loglift  logprob
351  Default  136.000000                    nan  136.000000  30.0000  30.0000
294  Default   33.000000                 london   33.000000  29.0000  29.0000
167  Default   41.000000                   epic   41.000000  28.0000  28.0000
561  Default   25.000000                   user   25.000000  27.0000  27.0000
599  Default   46.000000                   uxpa   46.000000  26.0000  26.0000
428  Default  164.000000               research  164.000000  25.0000  25.0000
84   Default   18.000000                    chi   18.000000  24.0000  24.0000
517  Default   21.000000                 strive   21.000000  23.0000  23.0000
102  Default  130.000000             conference  130.000000  22.0000  22.0000
608  Default   15.000000                    uxr   15.000000  21.0000  21.0000
562  Default   21.000000          user research   21.000000  20.0000  20.0000
312  Default   31.000000                 meetup   31.000000  19.0000  19.0000
277  Default   65.000000                  local   65.000000  18.0000  18.0000
437  Default   16.000000        research london   16.000000  17.0000  17.0000
550  Default   10.000000                toronto   10.000000  16.0000  16.0000
563  Default   15.000000   user research london   15.000000  15.0000  15.0000
567  Default  131.000000                     ux  131.000000  14.0000  14.0000
531  Default   20.000000                 summit   20.000000  13.0000  13.0000
136  Default   24.000000                    day   24.000000  12.0000  12.0000
282  Default   20.000000          local meetups   20.000000  11.0000  11.0000
195  Default   12.000000                focused   12.000000  10.0000  10.0000
176  Default   46.000000                  event   46.000000   9.0000   9.0000
638  Default   20.000000                   year   20.000000   8.0000   8.0000
142  Default   58.000000                 design   58.000000   7.0000   7.0000
37   Default   22.000000               attended   22.000000   6.0000   6.0000
518  Default    6.000000      strive conference    6.000000   5.0000   5.0000
390  Default    9.000000                 people    9.000000   4.0000   4.0000
631  Default    9.000000                  world    9.000000   3.0000   3.0000
46   Default   12.000000              australia   12.000000   2.0000   2.0000
625  Default   50.000000               workshop   50.000000   1.0000   1.0000
..       ...         ...                    ...         ...      ...      ...
263   Topic5    2.636865          jan chipchase    3.325249   1.9615  -5.0662
123   Topic5    2.636256        conference year    3.325099   1.9613  -5.0665
602   Topic5    1.813918               uxpa chi    2.500634   1.8724  -5.4403
197   Topic5    1.813916                  forum    2.500634   1.8724  -5.4403
86    Topic5    1.813914         chi conference    2.500634   1.8724  -5.4403
529   Topic5    1.813912                  stuff    2.500634   1.8724  -5.4403
466   Topic5    1.813859                  sadly    2.500636   1.8723  -5.4404
87    Topic5    1.813551               chi uxpa    2.500626   1.8722  -5.4405
177   Topic5    1.813473            event apart    2.500625   1.8721  -5.4406
29    Topic5    1.813473                  apart    2.500625   1.8721  -5.4406
488   Topic5    1.813309                  small    2.500666   1.8720  -5.4407
520   Topic5    1.813207         strive toronto    2.500618   1.8720  -5.4407
609   Topic5    1.812369         uxr collective    2.500731   1.8715  -5.4412
452   Topic5    1.811649  research thing meetup    2.500786   1.8711 

#### Topic groups 

1. UX
2. Design
3. Epic
4. User research
5. Strive



In [104]:
lda_W16 = lda16.transform(events_attend_recent_matrix)

In [105]:
top_doc_column16 = datadf.events_attend_recent.dropna()

In [106]:
word_count_matrix16, count_vect16 = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [108]:
LDA16a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA16a.fit(word_count_matrix16)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [109]:
lda_H = LDA16a.transform(word_count_matrix16)

In [110]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column16, 3)

Top 3 Documents for Topic 0: 

Document 1
uxr collective's strive conference in toronto

Document 2
nux manchester steve portigal's research networking dinner meetup at answerlab how to minimize research bias in your project

Document 3
strive local uxpa chapter meeting boston uxpa conference

Top 3 Documents for Topic 1: 

Document 1
researchops workshop user research sydney user research london ux insight utrecht

Document 2
research ops remote meetups ux monday in prague research tuesday in prague some ux conference meetups and barcamps in berlin bratislava brno prague and vienna

Document 3
not really research only but when i go to ux or web conference i pick research speaches nng exconf bratislava doer budapest webexpo prague ux salon tel aviv ux camp europe in berlin best thing ever mostly and i have a ticket for ux brighton

Top 3 Documents for Topic 2: 

Document 1
a 2 day ux design and research course numerous monthly nux event an nux conference in manchester

Document 2
actua

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [ ]:
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.basic_clean)
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.lemmatize)

In [170]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), ngram=(1,3), max_df=.3)
other_conference_types_matrix, other_conference_types_vector

(<258x300 sparse matrix of type '<class 'numpy.int64'>'
 	with 1103 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [174]:
lda20 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [175]:
lda20.fit(other_conference_types_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [176]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.095212  0.0       1        1  54.547071
1     -0.095212  0.0       2        1  45.452929, topic_info=    Category       Freq             Term      Total  loglift  logprob
158  Default  15.000000             nope  15.000000  30.0000  30.0000
217  Default  23.000000         sessions  23.000000  29.0000  29.0000
295  Default  17.000000        workshops  17.000000  28.0000  28.0000
259  Default  18.000000            think  18.000000  27.0000  27.0000
31   Default  13.000000      conferences  13.000000  26.0000  26.0000
216  Default  10.000000          session  10.000000  25.0000  25.0000
18   Default   7.000000             case   7.000000  24.0000  24.0000
156  Default   7.000000              new   7.000000  23.0000  23.0000
178  Default   6.000000           poster   6.000000  22.0000  22.0000
19   Default   6.000000     case studies   6.000000  21.0000  21.0000
243  Default   6.000000          studies   6.000000  20.0000  20.0000
53   Default   6.000000       discussion   6.000000  19.0000  19.0000
229  Default   5.000000            small   5.000000  18.0000  18.0000
267  Default   8.000000            track   8.000000  17.0000  17.0000
167  Default   4.000000           panels   4.000000  16.0000  16.0000
127  Default   4.000000           larger   4.000000  15.0000  15.0000
202  Default  18.000000         research  18.000000  14.0000  14.0000
43   Default   6.000000              day   6.000000  13.0000  13.0000
112  Default   5.000000         industry   5.000000  12.0000  12.0000
49   Default   5.000000           design   5.000000  11.0000  11.0000
180  Default   3.000000  poster sessions   3.000000  10.0000  10.0000
258  Default   3.000000           things   3.000000   9.0000   9.0000
165  Default   3.000000            panel   3.000000   8.0000   8.0000
47   Default   3.000000            demos   3.000000   7.0000   7.0000
139  Default   5.000000             long   5.000000   6.0000   6.0000
12   Default   3.000000             away   3.000000   5.0000   5.0000
248  Default   8.000000             talk   8.000000   4.0000   4.0000
118  Default   5.000000      interesting   5.000000   3.0000   3.0000
66   Default   8.000000           events   8.000000   2.0000   2.0000
32   Default   4.000000          content   4.000000   1.0000   1.0000
..       ...        ...              ...        ...      ...      ...
208   Topic2   2.724988          retreat   3.133762   0.6487  -5.2874
289   Topic2   2.724078             ways   3.133804   0.6484  -5.2877
132   Topic2   2.723673          learned   3.133822   0.6482  -5.2879
285   Topic2   2.720085          virtual   3.133988   0.6468  -5.2892
157   Topic2   2.718384             nice   3.134066   0.6462  -5.2898
290   Topic2   2.716682         webinars   3.134145   0.6455  -5.2904
163   Topic2   2.712074           option   3.134357   0.6438  -5.2921
191   Topic2   2.706777          project   3.134601   0.6417  -5.2941
27    Topic2   2.696937           common   3.135054   0.6380  -5.2977
213   Topic2   2.685199              say   3.135595   0.6334  -5.3021
147   Topic2   2.683242        mentioned   3.135686   0.6327  -5.3028
217   Topic2  19.936012         sessions  23.526318   0.6229  -3.2973
22    Topic2   2.653909             come   3.137037   0.6212  -5.3138
251   Topic2   1.946599             team   2.354798   0.5981  -5.6238
118   Topic2   4.344214      interesting   5.503645   0.5519  -4.8210
202   Topic2  12.077477         research  18.184610   0.3793  -3.7985
66    Topic2   6.160834           events   8.679263   0.4458  -4.4716
28    Topic2   9.875397       conference  17.471248   0.2180  -3.9998
141   Topic2   4.878699             make   7.108679   0.4121  -4.7050
172   Topic2  11.404994           people  24.734252   0.0144  -3.8558
92    Topic2   4.348133            group   6.318296   0.4148  -4.8201
197   Topic2   4.258562             real   6.322423

#### Topic groups 

1. Industry
2. Case studies



In [80]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [81]:
top_doc_column20 = datadf.other_conference_types.dropna()

In [82]:
word_count_matrix20, count_vect20 = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [83]:
LDA20a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA20a.fit(word_count_matrix20)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [84]:
lda_H = LDA20a.transform(word_count_matrix20)

In [85]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column20, 3)

Top 3 Documents for Topic 0: 

Document 1
Workshops in their original definition where participants brought their own problems and worked through them together, with help from a facilitator. NOT modern workshops that are tutorials rather than workshops. 

I deeply crave (and so does the list about) in-depth problem-solving interactive sessions. Everything else is so NOT important for where I am in my career (UX Leadership level).  

Document 2
It's not so much about the format for me as the content. I thought the IA Summit and XOXO and Eyeo had the best program ideas and mix in my own experience. Eyeo was dual track. Smallish, focused, lots of time to meet people, a wide variety of speakers and interesting talks

Document 3
"Fuller"-length talks: 30 minutes is not a "full talk." 60 rarely provides much depth unless the domain is focused. Workshops often include activities just for the sake of keeping people engaged. I'd eagerly attend a 2-4 hour talk (with a break) if the speaker was e

## 22. If attending a conference about research, who might you be excited to see there?

In [86]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [177]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.3)
ideal_att_matrix, ideal_att_vector

(<471x923 sparse matrix of type '<class 'numpy.int64'>'
 	with 4277 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [178]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [179]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [90]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.122573  0.065619       1        1  46.076154
2      0.007800 -0.120126       2        1  28.072781
1     -0.130373  0.054507       3        1  25.851065, topic_info=    Category        Freq                  Term       Total  loglift  logprob
668  Default  179.000000              research  179.000000  30.0000  30.0000
557  Default  188.000000                people  188.000000  29.0000  29.0000
514  Default   18.000000                   new   18.000000  28.0000  28.0000
708  Default   15.000000            sam ladner   15.000000  27.0000  27.0000
406  Default   15.000000                ladner   15.000000  26.0000  26.0000
707  Default   15.000000                   sam   15.000000  25.0000  25.0000
775  Default   13.000000                 spool   13.000000  24.0000  24.0000
922  Default   16.000000                 young   16.000000  23.0000  23.0000
380  Default   12.000000           jared spool   12.000000  22.0000  22.0000
195  Default   26.000000                 doing   26.000000  21.0000  21.0000
348  Default   12.000000                  indi   12.000000  20.0000  20.0000
349  Default   12.000000            indi young   12.000000  19.0000  19.0000
836  Default   11.000000                 topic   11.000000  18.0000  18.0000
379  Default   13.000000                 jared   13.000000  17.0000  17.0000
786  Default   22.000000                 steve   22.000000  16.0000  16.0000
4    Default   24.000000              academic   24.000000  15.0000  15.0000
353  Default   41.000000              industry   41.000000  14.0000  14.0000
602  Default   18.000000              portigal   18.000000  13.0000  13.0000
788  Default   17.000000        steve portigal   17.000000  12.0000  12.0000
532  Default    8.000000                norman    8.000000  11.0000  11.0000
308  Default   15.000000                  hall   15.000000  10.0000  10.0000
303  Default   12.000000                 group   12.000000   9.0000   9.0000
853  Default   12.000000                  type   12.000000   8.0000   8.0000
190  Default   13.000000             diversity   13.000000   7.0000   7.0000
811  Default   20.000000                  team   20.000000   6.0000   6.0000
422  Default    8.000000          leader field    8.000000   5.0000   5.0000
226  Default   14.000000                 erika   14.000000   4.0000   4.0000
158  Default    9.000000                   day    9.000000   3.0000   3.0000
547  Default   11.000000                  orgs   11.000000   2.0000   2.0000
889  Default   26.000000                  want   26.000000   1.0000   1.0000
..       ...         ...                   ...         ...      ...      ...
288   Topic3    3.527255               goodwin    4.112702   1.1993  -5.8859
397   Topic3    3.527255           kim goodwin    4.112702   1.1993  -5.8859
693   Topic3    3.526336  researcher different    4.112350   1.1991  -5.8862
735   Topic3    3.522396               sharing    4.113478   1.1977  -5.8873
732   Topic3    3.511522                   set    4.113507   1.1946  -5.8904
188   Topic3    3.503869         diverse group    4.114081   1.1923  -5.8925
499   Topic3    4.188803                 mixed    4.932974   1.1893  -5.7140
832   Topic3    6.814088                 tomer    8.213368   1.1660  -5.2274
158   Topic3    7.339972                   day    9.089237   1.1391  -5.1531
736   Topic3    5.935992                sharon    7.394905   1.1331  -5.3654
303   Topic3    9.038417                 group   12.323398   1.0428  -4.9449
824   Topic3    7.718688               thought   12.431304   0.8762  -5.1028
785   Topic3    5.540375               startup    8.224825   0.9577  -5.4344
786   Topic3   10.494819                 steve   22.213930   0.6030  -4.7955
439   Topic3    4.749443                 leisa    6.578592   1.0270  -5.5884
692   Topic3   19.493240            researcher   82.210862  -0.0864 

#### Topic groups 

1. Industry
2. Indi Young
3. Sam Ladner



In [91]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [92]:
top_doc_column22 = datadf.ideal_attendees.dropna()

In [93]:
word_count_matrix22, count_vect22 = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [94]:
LDA22a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA22a.fit(word_count_matrix22)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [95]:
lda_H = LDA22a.transform(word_count_matrix22)

In [96]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column22, 3)

Top 3 Documents for Topic 0: 

Document 1
people with experience in building company by applying research knowledge i think there are people out there speaking from a consulting view they donat take a many risk a the one iad like to learn from company are usually led by people with a techdesign background if research is creating a real value in company and if we believe research can lead the strategic vision of a company iall be excited to see people who have managed to achieve that

Document 2
i would like to hear from and network with other organizational leader i want to meet people i can learn from no pr or salestype presentation no product pushing although i'd like to learn about tool that people are using i am much more interested in strategy and how to manage operation for large team also speaking a someone on the consulting side it would be nice to hear from leader in that space a well a leader within organization some of the advice from design executive doesn't really apply to